# 📊 TourismIQ - Notebook 1: Data Collection & EDA

**Objectif**: Explorer les données collectées et préparer le terrain pour le Quality Scorer

## Données disponibles
- **DATAtourisme**: 50k POIs (labels, descriptions, géoloc, types)
- **Opendatasoft**: 10k communes (population, superficie, densité)
- **Open-Meteo**: 13 régions (climat, température, précipitations)

## Questions clés
1. Quelle est la structure des POIs DATAtourisme ?
2. Quel est le taux de complétude des champs importants ?
3. Quelle est la distribution géographique des POIs ?
4. Quels types de POIs avons-nous ?
5. Quelles features utiliser pour le Quality Scorer ?

In [ ]:
# Imports
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Config viz
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print("✅ Imports OK")

## 1. Chargement des données

In [ ]:
# Chemins des fichiers
DATA_DIR = Path("../data/raw")

pois_file = DATA_DIR / "datatourisme_pois_50k.parquet"
communes_file = DATA_DIR / "communes_population_all.parquet"
climate_file = DATA_DIR / "climate_regions.parquet"

# Vérifier existence
print("📁 Fichiers disponibles:")
print(f"  POIs: {pois_file.exists()} ({pois_file.stat().st_size / 1024 / 1024:.2f} MB)" if pois_file.exists() else "  POIs: ❌")
print(f"  Communes: {communes_file.exists()} ({communes_file.stat().st_size / 1024:.2f} KB)" if communes_file.exists() else "  Communes: ❌")
print(f"  Climat: {climate_file.exists()} ({climate_file.stat().st_size / 1024:.2f} KB)" if climate_file.exists() else "  Climat: ❌")

In [ ]:
# Charger POIs
print("📂 Chargement des POIs...")
df_pois = pd.read_parquet(pois_file)
print(f"✅ {len(df_pois):,} POIs chargés")
print(f"   Colonnes: {len(df_pois.columns)}")
print(f"   Mémoire: {df_pois.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

In [ ]:
# Charger communes
print("📂 Chargement des communes...")
df_communes = pd.read_parquet(communes_file)
print(f"✅ {len(df_communes):,} communes chargées")
print(f"   Colonnes: {list(df_communes.columns)}")

In [ ]:
# Charger climat
print("📂 Chargement du climat...")
df_climate = pd.read_parquet(climate_file)
print(f"✅ {len(df_climate)} régions chargées")
print(f"   Colonnes: {list(df_climate.columns)}")
df_climate.head()

## 2. Structure des POIs DATAtourisme

In [ ]:
# Aperçu général
print("📊 Informations générales:")
print(f"  Shape: {df_pois.shape}")
print(f"  Colonnes: {len(df_pois.columns)}")
print(f"\n📋 Liste des colonnes:")
for i, col in enumerate(df_pois.columns, 1):
    dtype = df_pois[col].dtype
    non_null = df_pois[col].notna().sum()
    pct = non_null / len(df_pois) * 100
    print(f"  {i:2d}. {col:30s} - {str(dtype):10s} - {non_null:6,} ({pct:5.1f}%) non-null")

In [ ]:
# Échantillon de données
print("🔍 Échantillon (premier POI):")
df_pois.iloc[0]

## 3. Analyse de Complétude

Pour le Quality Scorer, on doit évaluer la complétude des champs importants.

In [ ]:
# Champs clés pour le Quality Score
key_fields = [
    '@id',           # ID unique
    '@type',         # Type de POI
    'rdfs:label',    # Nom/label
    'hasDescription',# Description
    'isLocatedAt',   # Localisation
    'hasContact',    # Contact
    'hasBeenCreatedBy', # Créateur/source
    'hasBeenPublishedBy', # Éditeur
]

print("📊 Complétude des champs clés:")
print("\n" + "="*60)
completeness = {}
for field in key_fields:
    if field in df_pois.columns:
        non_null = df_pois[field].notna().sum()
        pct = non_null / len(df_pois) * 100
        completeness[field] = pct
        print(f"{field:25s}: {non_null:6,} / {len(df_pois):6,} ({pct:5.1f}%)")
    else:
        print(f"{field:25s}: ABSENT")
print("="*60)

In [ ]:
# Visualisation complétude
fig, ax = plt.subplots(figsize=(12, 6))
fields = list(completeness.keys())
values = list(completeness.values())

colors = ['green' if v > 80 else 'orange' if v > 50 else 'red' for v in values]
bars = ax.barh(fields, values, color=colors, alpha=0.7)

ax.set_xlabel('Complétude (%)', fontsize=12)
ax.set_title('Complétude des Champs Clés - DATAtourisme POIs', fontsize=14, fontweight='bold')
ax.set_xlim(0, 100)
ax.axvline(x=80, color='green', linestyle='--', alpha=0.3, label='Excellent (>80%)')
ax.axvline(x=50, color='orange', linestyle='--', alpha=0.3, label='Moyen (>50%)')
ax.grid(axis='x', alpha=0.3)
ax.legend()

# Ajouter valeurs
for i, (field, value) in enumerate(zip(fields, values)):
    ax.text(value + 2, i, f'{value:.1f}%', va='center', fontsize=10)

plt.tight_layout()
plt.show()

## 4. Extraction et Parsing des Champs Complexes

Les champs DATAtourisme sont souvent en JSON (nested). On doit les parser.

In [ ]:
# Helper: parser JSON strings
def parse_json_field(value):
    """Parse un champ qui peut être string JSON ou déjà parsé"""
    if pd.isna(value):
        return None
    if isinstance(value, str):
        try:
            return json.loads(value)
        except:
            return value
    return value

print("✅ Helper parse_json_field défini")

In [ ]:
# Exemple: parser @type (type de POI)
print("🔍 Analyse du champ @type:")
print("\nExemples de valeurs:")
for i in range(min(5, len(df_pois))):
    type_val = df_pois['@type'].iloc[i] if '@type' in df_pois.columns else None
    print(f"  {i+1}. {type_val}")

In [ ]:
# Extraire types principaux
def extract_main_type(type_val):
    """Extrait le type principal d'un POI"""
    type_val = parse_json_field(type_val)
    if isinstance(type_val, list):
        # Prendre le type le plus spécifique (souvent le premier non-générique)
        for t in type_val:
            if t not in ['schema:Thing', 'schema:Place', 'olo:OrderedList']:
                return t
        return type_val[0] if type_val else None
    return type_val

if '@type' in df_pois.columns:
    df_pois['type_principal'] = df_pois['@type'].apply(extract_main_type)
    print("✅ Colonne 'type_principal' créée")
    print(f"\n📊 Distribution des types (top 20):")
    print(df_pois['type_principal'].value_counts().head(20))
else:
    print("⚠️  Colonne @type absente")

In [ ]:
# Visualiser distribution des types
if 'type_principal' in df_pois.columns:
    top_types = df_pois['type_principal'].value_counts().head(15)
    
    fig, ax = plt.subplots(figsize=(12, 8))
    top_types.plot(kind='barh', ax=ax, color='steelblue', alpha=0.7)
    ax.set_xlabel('Nombre de POIs', fontsize=12)
    ax.set_title('Top 15 Types de POIs - DATAtourisme', fontsize=14, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    
    # Ajouter valeurs
    for i, v in enumerate(top_types.values):
        ax.text(v + 50, i, f'{v:,}', va='center', fontsize=10)
    
    plt.tight_layout()
    plt.show()

## 5. Extraction GPS & Distribution Géographique

In [ ]:
# Extraire coordonnées GPS
def extract_coordinates(located_at):
    """Extrait latitude, longitude depuis isLocatedAt"""
    located_at = parse_json_field(located_at)
    if not located_at or not isinstance(located_at, list):
        return None, None
    
    for location in located_at:
        if isinstance(location, dict) and 'schema:geo' in location:
            geo = location['schema:geo']
            if isinstance(geo, dict):
                lat = geo.get('schema:latitude')
                lon = geo.get('schema:longitude')
                if lat and lon:
                    try:
                        return float(lat), float(lon)
                    except:
                        pass
    return None, None

print("🗺️  Extraction des coordonnées GPS...")
if 'isLocatedAt' in df_pois.columns:
    coords = df_pois['isLocatedAt'].apply(extract_coordinates)
    df_pois['latitude'] = coords.apply(lambda x: x[0])
    df_pois['longitude'] = coords.apply(lambda x: x[1])
    
    # Stats
    pois_with_coords = df_pois['latitude'].notna().sum()
    print(f"✅ POIs avec coordonnées: {pois_with_coords:,} / {len(df_pois):,} ({pois_with_coords/len(df_pois)*100:.1f}%)")
else:
    print("⚠️  Colonne isLocatedAt absente")

In [ ]:
# Distribution géographique
if 'latitude' in df_pois.columns and df_pois['latitude'].notna().sum() > 0:
    df_geo = df_pois[df_pois['latitude'].notna()].copy()
    
    print(f"📍 Distribution géographique ({len(df_geo):,} POIs):")
    print(f"  Latitude: {df_geo['latitude'].min():.4f} à {df_geo['latitude'].max():.4f}")
    print(f"  Longitude: {df_geo['longitude'].min():.4f} à {df_geo['longitude'].max():.4f}")
    
    # Scatter plot
    fig, ax = plt.subplots(figsize=(14, 10))
    
    # Densité avec hexbin
    hb = ax.hexbin(df_geo['longitude'], df_geo['latitude'], 
                   gridsize=50, cmap='YlOrRd', alpha=0.7, mincnt=1)
    
    ax.set_xlabel('Longitude', fontsize=12)
    ax.set_ylabel('Latitude', fontsize=12)
    ax.set_title('Distribution Géographique des POIs en France', fontsize=14, fontweight='bold')
    ax.grid(alpha=0.3)
    
    # Colorbar
    cb = plt.colorbar(hb, ax=ax)
    cb.set_label('Densité de POIs', fontsize=11)
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️  Pas de coordonnées GPS disponibles")

## 6. Extraction Descriptions & Analyse Textuelle

In [ ]:
# Extraire description FR
def extract_description_fr(desc_list):
    """Extrait description courte en français"""
    desc_list = parse_json_field(desc_list)
    if not desc_list or not isinstance(desc_list, list):
        return None
    
    for desc in desc_list:
        if isinstance(desc, dict) and 'shortDescription' in desc:
            short_desc = desc['shortDescription']
            if isinstance(short_desc, dict):
                # Essayer plusieurs clés pour le français
                return short_desc.get('fr') or short_desc.get('@fr') or short_desc.get('fr-FR')
            elif isinstance(short_desc, str):
                return short_desc
    return None

print("📝 Extraction des descriptions...")
if 'hasDescription' in df_pois.columns:
    df_pois['description'] = df_pois['hasDescription'].apply(extract_description_fr)
    
    # Stats
    pois_with_desc = df_pois['description'].notna().sum()
    print(f"✅ POIs avec description: {pois_with_desc:,} / {len(df_pois):,} ({pois_with_desc/len(df_pois)*100:.1f}%)")
    
    # Longueur descriptions
    df_pois['description_length'] = df_pois['description'].fillna('').str.len()
    
    print(f"\n📊 Statistiques longueur descriptions:")
    print(df_pois['description_length'].describe())
else:
    print("⚠️  Colonne hasDescription absente")

In [ ]:
# Distribution longueur descriptions
if 'description_length' in df_pois.columns:
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Histogramme
    desc_lengths = df_pois[df_pois['description_length'] > 0]['description_length']
    axes[0].hist(desc_lengths, bins=50, color='steelblue', alpha=0.7, edgecolor='black')
    axes[0].set_xlabel('Longueur (caractères)', fontsize=11)
    axes[0].set_ylabel('Nombre de POIs', fontsize=11)
    axes[0].set_title('Distribution Longueur Descriptions', fontsize=12, fontweight='bold')
    axes[0].axvline(desc_lengths.median(), color='red', linestyle='--', label=f'Médiane: {desc_lengths.median():.0f}')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    # Boxplot
    axes[1].boxplot(desc_lengths, vert=True)
    axes[1].set_ylabel('Longueur (caractères)', fontsize=11)
    axes[1].set_title('Boxplot Longueur Descriptions', fontsize=12, fontweight='bold')
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Exemples de descriptions
if 'description' in df_pois.columns:
    print("📝 Exemples de descriptions:\n")
    print("="*80)
    
    sample_indices = df_pois[df_pois['description'].notna()].sample(min(3, len(df_pois))).index
    
    for i, idx in enumerate(sample_indices, 1):
        row = df_pois.loc[idx]
        nom = row.get('rdfs:label', 'N/A')
        if isinstance(nom, dict):
            nom = nom.get('fr', nom.get('@fr', 'N/A'))
        desc = row['description']
        length = len(desc) if desc else 0
        
        print(f"\n{i}. {nom}")
        print(f"   Type: {row.get('type_principal', 'N/A')}")
        print(f"   Longueur: {length} caractères")
        print(f"   Description: {desc[:200]}..." if length > 200 else f"   Description: {desc}")
        print("-"*80)

## 7. Features pour le Quality Scorer

Identifier les features importantes pour scorer la qualité des POIs.

In [ ]:
# Calculer features de complétude
print("⚙️  Calcul des features de qualité...\n")

features_quality = {}

# 1. Has name/label
if 'rdfs:label' in df_pois.columns:
    features_quality['has_name'] = df_pois['rdfs:label'].notna().astype(int)
    print(f"✅ has_name: {features_quality['has_name'].sum():,} / {len(df_pois):,}")

# 2. Has description
if 'description' in df_pois.columns:
    features_quality['has_description'] = df_pois['description'].notna().astype(int)
    features_quality['description_length'] = df_pois['description_length']
    print(f"✅ has_description: {features_quality['has_description'].sum():,} / {len(df_pois):,}")
    print(f"✅ description_length: moyenne {features_quality['description_length'].mean():.1f} caractères")

# 3. Has GPS
if 'latitude' in df_pois.columns:
    features_quality['has_gps'] = df_pois['latitude'].notna().astype(int)
    print(f"✅ has_gps: {features_quality['has_gps'].sum():,} / {len(df_pois):,}")

# 4. Has type
if 'type_principal' in df_pois.columns:
    features_quality['has_type'] = df_pois['type_principal'].notna().astype(int)
    print(f"✅ has_type: {features_quality['has_type'].sum():,} / {len(df_pois):,}")

# 5. Has contact
if 'hasContact' in df_pois.columns:
    features_quality['has_contact'] = df_pois['hasContact'].notna().astype(int)
    print(f"✅ has_contact: {features_quality['has_contact'].sum():,} / {len(df_pois):,}")

print(f"\n📊 Total features calculées: {len(features_quality)}")

In [ ]:
# Score de complétude simple (baseline)
df_features = pd.DataFrame(features_quality)

# Score simplifié (0-100)
df_features['completeness_score_simple'] = (
    df_features['has_name'] * 20 +
    df_features['has_description'] * 30 +
    df_features['has_gps'] * 20 +
    df_features['has_type'] * 15 +
    df_features['has_contact'] * 15
)

print("🎯 Score de complétude (baseline):")
print(f"  Moyenne: {df_features['completeness_score_simple'].mean():.1f} / 100")
print(f"  Médiane: {df_features['completeness_score_simple'].median():.1f} / 100")
print(f"  Écart-type: {df_features['completeness_score_simple'].std():.1f}")
print(f"\n📊 Distribution:")
print(df_features['completeness_score_simple'].describe())

In [ ]:
# Visualiser distribution score
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogramme
axes[0].hist(df_features['completeness_score_simple'], bins=20, 
             color='steelblue', alpha=0.7, edgecolor='black')
axes[0].set_xlabel('Score de Complétude (0-100)', fontsize=11)
axes[0].set_ylabel('Nombre de POIs', fontsize=11)
axes[0].set_title('Distribution Score de Complétude (Baseline)', fontsize=12, fontweight='bold')
axes[0].axvline(df_features['completeness_score_simple'].mean(), 
                color='red', linestyle='--', label=f"Moyenne: {df_features['completeness_score_simple'].mean():.1f}")
axes[0].legend()
axes[0].grid(alpha=0.3)

# Boxplot par qualité
score_categories = pd.cut(df_features['completeness_score_simple'], 
                          bins=[0, 40, 60, 80, 100], 
                          labels=['Low (<40)', 'Medium (40-60)', 'Good (60-80)', 'Excellent (80-100)'])

category_counts = score_categories.value_counts().sort_index()
category_counts.plot(kind='bar', ax=axes[1], color=['red', 'orange', 'lightgreen', 'green'], alpha=0.7)
axes[1].set_xlabel('Catégorie de Qualité', fontsize=11)
axes[1].set_ylabel('Nombre de POIs', fontsize=11)
axes[1].set_title('Répartition par Catégorie de Qualité', fontsize=12, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)
axes[1].tick_params(axis='x', rotation=45)

# Ajouter valeurs
for i, v in enumerate(category_counts.values):
    axes[1].text(i, v + 100, f'{v:,}\n({v/len(df_features)*100:.1f}%)', 
                 ha='center', fontsize=9)

plt.tight_layout()
plt.show()

## 8. Analyse Démographique (Communes)

In [ ]:
# Overview communes
print("🏘️  Données Communes (Opendatasoft):\n")
print(f"  Total communes: {len(df_communes):,}")
print(f"  Population totale: {df_communes['population'].sum():,.0f}")
print(f"  Population moyenne: {df_communes['population'].mean():,.0f}")
print(f"\n📊 Top 10 communes par population:")
print(df_communes.nlargest(10, 'population')[['nom_commune', 'population', 'densite_hab_km2']])

In [ ]:
# Distribution population
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogramme (échelle log)
axes[0].hist(df_communes['population'], bins=50, color='steelblue', alpha=0.7, edgecolor='black')
axes[0].set_xlabel('Population', fontsize=11)
axes[0].set_ylabel('Nombre de communes', fontsize=11)
axes[0].set_title('Distribution Population Communes', fontsize=12, fontweight='bold')
axes[0].set_yscale('log')
axes[0].grid(alpha=0.3)

# Densité
axes[1].hist(df_communes['densite_hab_km2'], bins=50, color='coral', alpha=0.7, edgecolor='black')
axes[1].set_xlabel('Densité (hab/km²)', fontsize=11)
axes[1].set_ylabel('Nombre de communes', fontsize=11)
axes[1].set_title('Distribution Densité Communes', fontsize=12, fontweight='bold')
axes[1].set_yscale('log')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Analyse Climat (Régions)

In [ ]:
# Overview climat
print("🌤️  Données Climatiques (Open-Meteo):\n")
print(df_climate[['region', 'temp_avg_annual', 'precipitation_annual_mm', 'climate_type']].to_string(index=False))

In [ ]:
# Visualisation climat
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Température moyenne par région
df_climate_sorted = df_climate.sort_values('temp_avg_annual')
axes[0, 0].barh(df_climate_sorted['region'], df_climate_sorted['temp_avg_annual'], 
                color='orangered', alpha=0.7)
axes[0, 0].set_xlabel('Température Moyenne Annuelle (°C)', fontsize=10)
axes[0, 0].set_title('Température par Région', fontsize=11, fontweight='bold')
axes[0, 0].grid(axis='x', alpha=0.3)

# 2. Précipitations par région
df_climate_sorted = df_climate.sort_values('precipitation_annual_mm')
axes[0, 1].barh(df_climate_sorted['region'], df_climate_sorted['precipitation_annual_mm'], 
                color='steelblue', alpha=0.7)
axes[0, 1].set_xlabel('Précipitations Annuelles (mm)', fontsize=10)
axes[0, 1].set_title('Précipitations par Région', fontsize=11, fontweight='bold')
axes[0, 1].grid(axis='x', alpha=0.3)

# 3. Distribution types climatiques
climate_counts = df_climate['climate_type'].value_counts()
climate_counts.plot(kind='bar', ax=axes[1, 0], color='seagreen', alpha=0.7)
axes[1, 0].set_xlabel('Type Climatique', fontsize=10)
axes[1, 0].set_ylabel('Nombre de Régions', fontsize=10)
axes[1, 0].set_title('Distribution Types Climatiques', fontsize=11, fontweight='bold')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(axis='y', alpha=0.3)

# 4. Scatter temp vs précipitations
colors_map = {'mediterranean': 'orangered', 'oceanic': 'steelblue', 
              'continental': 'gold', 'mountain': 'purple'}
for climate_type in df_climate['climate_type'].unique():
    data = df_climate[df_climate['climate_type'] == climate_type]
    axes[1, 1].scatter(data['temp_avg_annual'], data['precipitation_annual_mm'], 
                       label=climate_type, s=100, alpha=0.7, 
                       color=colors_map.get(climate_type, 'gray'))

axes[1, 1].set_xlabel('Température Moyenne (°C)', fontsize=10)
axes[1, 1].set_ylabel('Précipitations (mm)', fontsize=10)
axes[1, 1].set_title('Température vs Précipitations par Type', fontsize=11, fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Conclusions & Next Steps

### 📊 Résumé des Données

**POIs DATAtourisme:**
- 50k POIs collectés
- Complétude variable selon les champs (à améliorer pour le scoring)
- Distribution géographique couvre toute la France
- Types variés (hébergement, restauration, culture, activités)

**Communes & Climat:**
- 10k communes avec données démographiques
- 13 régions avec données climatiques
- Prêt pour enrichissement géographique

### 🎯 Features Identifiées pour Quality Scorer

**Complétude (40%):**
- has_name, has_description, has_gps, has_type, has_contact
- description_length

**Richesse Textuelle (30%):**
- description_richness (à calculer avec textstat)
- nb_languages (multilinguisme)

**Contexte Géo (20%):**
- commune_population, commune_density
- poi_density_radius (nb POIs similaires à proximité)
- climate_type

**Freshness (10%):**
- last_update_days (si disponible)

### ⏭️ Next Steps

1. **Feature Engineering** (Notebook 02):
   - Calculer les 20 features du Quality Scorer
   - Enrichir avec données communes/climat
   - Créer target synthétique (score 0-100)

2. **Quality Scorer Training** (Notebook 03):
   - Entraîner LightGBM/XGBoost
   - Tuning hyperparamètres
   - Validation (target R² > 0.75)

3. **Gap Detection** (Notebook 04):
   - HDBSCAN clustering
   - Analyse distribution types
   - Random Forest opportunités

In [ ]:
# Sauvegarder version enrichie pour le prochain notebook
print("💾 Sauvegarde des données enrichies...")

# Combiner POIs avec features de base
df_pois_enriched = df_pois.copy()
for col in df_features.columns:
    df_pois_enriched[col] = df_features[col]

output_file = Path("../data/processed/pois_enriched_eda.parquet")
output_file.parent.mkdir(parents=True, exist_ok=True)

df_pois_enriched.to_parquet(output_file, index=False, compression='snappy')

print(f"✅ Sauvegardé: {output_file}")
print(f"   Records: {len(df_pois_enriched):,}")
print(f"   Colonnes: {len(df_pois_enriched.columns)}")
print(f"   Taille: {output_file.stat().st_size / 1024 / 1024:.2f} MB")

In [ ]:
print("\n" + "="*80)
print("✅ NOTEBOOK 1 - EDA TERMINÉ")
print("="*80)
print("\n📈 Prochaine étape: Notebook 02 - Feature Engineering pour Quality Scorer")
print("\n🎯 Objectif: Créer les 20 features ML et le target score synthétique (0-100)")